In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
%load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
project_id = "perceptive-ivy-290216"

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [4]:
query2=f"""
SELECT
A.*,
B.RoundNumber
FROM `perceptive-ivy-290216.f1_api.results_race` A
INNER JOIN
`perceptive-ivy-290216.f1_api.schedules` B
ON A.GP=B.EventName AND A.Year=B.Year
WHERE
A.Year=2022
AND B.Year=2022
-- AND
--  A.GP='Qatar Grand Prix'
"""

track2=pd.read_gbq(query2,project_id,dialect='standard')

In [5]:
query3=f"""
SELECT
A.*,
B.RoundNumber
FROM `perceptive-ivy-290216.f1_api.results_sprint` A
INNER JOIN
`perceptive-ivy-290216.f1_api.schedules` B
ON A.GP=B.EventName AND A.Year=B.Year
WHERE
A.Year=2022
-- AND
-- A.GP='Qatar Grand Prix'
"""
track3=pd.read_gbq(query3,project_id,dialect='standard')

In [6]:
#Points

In [149]:
points_table=track2[["RoundNumber","GP","DriverId","Points"]]
points_table["Points"]=points_table["Points"].astype(int)
points_table=points_table.sort_values(by=["RoundNumber","DriverId"])

<ipython-input-149-cda782badb73>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [150]:
points_table_sprint=track3[["RoundNumber","GP","DriverId","Points"]]
points_table_sprint["Points"]=points_table_sprint["Points"].astype(int)
points_table_sprint=points_table_sprint.sort_values(by=["RoundNumber","DriverId"])

<ipython-input-150-c7dfb14fd175>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [151]:
points_table=pd.concat([points_table,points_table_sprint])
points_table

,RoundNumber,GP,DriverId,Points
258,1,Bahrain Grand Prix,ALB,0
300,1,Bahrain Grand Prix,ALO,2
384,1,Bahrain Grand Prix,BOT,8
130,1,Bahrain Grand Prix,GAS,0
220,1,Bahrain Grand Prix,HAM,15
373,1,Bahrain Grand Prix,HUL,0
353,1,Bahrain Grand Prix,LAT,0
280,1,Bahrain Grand Prix,LEC,26
329,1,Bahrain Grand Prix,MAG,10
337,1,Bahrain Grand Prix,MSC,0


In [152]:
#Get all drivers GP counts to fill in missing data and remove drivers who drove sparingly. The Other Option is to add the GP Data for sparing drivers to keep the dataset uniform
points_table.groupby(by=["DriverId"])["GP"].count()

DriverId
ALB    24
ALO    25
BOT    25
DEV     1
GAS    25
HAM    25
HUL     2
LAT    25
LEC    25
MAG    25
MSC    24
NOR    25
OCO    25
PER    25
RIC    25
RUS    25
SAI    25
STR    25
TSU    25
VER    25
VET    23
ZHO    25
Name: GP, dtype: int64

In [153]:
#Find out which GPs were missed by drivers and then use the next block to fill in the details missing
# points_table[points_table["RoundNumber"].isin([16])].sort_values(by=["RoundNumber","DriverId"])
points_table[points_table["DriverId"]=="VET"]

,RoundNumber,GP,DriverId,Points
421,3,Australian Grand Prix,VET,0
427,4,Emilia Romagna Grand Prix,VET,4
438,5,Miami Grand Prix,VET,0
425,6,Spanish Grand Prix,VET,0
424,7,Monaco Grand Prix,VET,1
429,8,Azerbaijan Grand Prix,VET,8
432,9,Canadian Grand Prix,VET,0
433,10,British Grand Prix,VET,2
435,11,Austrian Grand Prix,VET,0
428,12,French Grand Prix,VET,0


In [154]:
points_table=points_table[(points_table["DriverId"]!="HUL")&(points_table["DriverId"]!="DEV")]

In [155]:
#Adding Albon's Italian GP Data which is missing. This messes up the line chart, so to avoid that and uniformity in data adding this row here.
points_table.loc[-1] = [16,"Italian Grand Prix", "ALB", 0] # adding a row
points_table.loc[-2] = [2,"Saudi Arabian Grand Prix", "MSC", 0] # adding a row
points_table.loc[-3] = [1,"Bahrain Grand Prix", "VET", 0] # adding a row
points_table.loc[-4] = [2,"Saudi Arabian Grand Prix	", "VET", 0] # adding a row

<ipython-input-155-a84d192bd034>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-155-a84d192bd034>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [160]:
#Combine Race and Sprint in 1 row
points_table=pd.DataFrame(points_table.groupby(by=["RoundNumber","GP","DriverId"])["Points"].sum()).reset_index()
points_table=points_table.sort_values(by=["DriverId","RoundNumber"])
points_table["cumsum"]=points_table.groupby(by=["DriverId"])["Points"].cumsum()

#Create a new table that has totals by driver id which will be used to sort the final chart
totals=pd.DataFrame(points_table.groupby(by=["DriverId"])["Points"].sum())
totals.columns=["sum"]

#Merge both tables
points_table=points_table.merge(totals, left_on='DriverId', right_on='DriverId')

points_table=points_table.sort_values(by=["sum","DriverId","RoundNumber"], ascending=[False, True, True])
points_table

,RoundNumber,GP,DriverId,Points,cumsum,sum
374,1,Bahrain Grand Prix,VER,0,0,454
375,2,Saudi Arabian Grand Prix,VER,25,25,454
376,3,Australian Grand Prix,VER,0,25,454
377,4,Emilia Romagna Grand Prix,VER,34,59,454
378,5,Miami Grand Prix,VER,26,85,454
379,6,Spanish Grand Prix,VER,25,110,454
380,7,Monaco Grand Prix,VER,15,125,454
381,8,Azerbaijan Grand Prix,VER,25,150,454
382,9,Canadian Grand Prix,VER,25,175,454
383,10,British Grand Prix,VER,6,181,454


In [194]:
fig=px.line(
    points_table,
    y="cumsum",
    x='GP',
    # text='cumsum',
    # text='LapTimeDelta2',
    color='DriverId',
    title="<b>Driver's Championship Points</b>",
    height=1200, width=1600    ,
    template="plotly_dark",


    color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
)
# for m in range(21):
#   fig.add_scatter(
#       x = [fig.data[m].x[-1]], y = [fig.data[m].y[-1]],
#           mode = 'text',
#           marker = {'color':'black', 'size':2},
#           showlegend = False,
#           text = [fig.data[m].y[-1]],
#           textposition='bottom right')
fig.show()

In [191]:
fig=px.scatter(
    points_table,
    y="DriverId",
    x='cumsum',
    color='DriverId',
    title="<b>Driver's Championship Points</b>",
    height=1000, width=1200,
    animation_frame="RoundNumber",
    animation_group="DriverId",
    range_x=[0,500],
    # range_y=[0,500],
    size='cumsum',
    # text='cumsum',

    color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
)

fig.show()

In [192]:
fig=px.bar(
    points_table,
    y="cumsum",
    x='DriverId',
    color='DriverId',
    title="<b>Driver's Championship Points</b>",
    height=1000, width=1200,
    animation_frame="RoundNumber",
    animation_group="DriverId",
    # range_x=[0,500],
    range_y=[0,500],
    template="plotly_dark",
    # size='cumsum',
    # text='cumsum',

     color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
)
fig.update_traces(textposition='outside')

fig.show()